# Stock Analysis: ORCL

This notebook analyzes volume patterns and inefficiencies for ORCL.

In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn google-cloud-bigquery pandas-gbq

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from datetime import datetime, timedelta

# Set plot style
sns.set(style="darkgrid")
plt.rcParams['figure.figsize'] = (14, 8)

In [ ]:
# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Configuration
TICKER = "ORCL"  # Will be replaced with actual ticker
PROJECT_ID = "delphi-449908"  # Will be replaced with actual project ID
DATASET = "trading_insights"
DIRECTION = "buy"  # Will be replaced with 'buy' or 'short'

In [ ]:
# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

In [ ]:
# Fetch data from BigQuery
query = f"""
SELECT *
FROM `delphi-449908.{DATASET}.stock_ORCL_prices`
ORDER BY date DESC
LIMIT 252  -- Approximately 1 year of trading days
"""

df = client.query(query).to_dataframe()

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Sort by date
df = df.sort_values('date')

# Display basic info
print(f"Loaded {len(df)} rows of data for ORCL")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
df.head()

## Volume Analysis

We'll analyze volume patterns to identify potential inefficiencies.

In [ ]:
# Calculate volume metrics
df['volume_ma20'] = df['volume'].rolling(window=20).mean()
df['volume_std20'] = df['volume'].rolling(window=20).std()
df['volume_ratio'] = df['volume'] / df['volume_ma20']
df['volume_z_score'] = (df['volume'] - df['volume_ma20']) / df['volume_std20']

# Identify volume spikes (Z-score > 2.0)
df['is_volume_spike'] = df['volume_z_score'] > 2.0

# Calculate price change
df['price_change'] = df['close'].pct_change() * 100

# Display summary
print(f"Number of volume spikes: {df['is_volume_spike'].sum()}")
df[df['is_volume_spike']].tail(5)

In [ ]:
# Visualize volume and price
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Plot price
ax1.plot(df['date'], df['close'], label='Close Price')
ax1.set_title(f'ORCL Price', fontsize=16)
ax1.set_ylabel('Price ($)', fontsize=14)
ax1.legend()
ax1.grid(True)

# Plot volume
ax2.bar(df['date'], df['volume'], color='blue', alpha=0.5, label='Volume')
ax2.plot(df['date'], df['volume_ma20'], color='red', label='20-day MA')

# Highlight volume spikes
spike_dates = df[df['is_volume_spike']]['date']
spike_volumes = df[df['is_volume_spike']]['volume']
ax2.scatter(spike_dates, spike_volumes, color='red', s=50, label='Volume Spike')

ax2.set_title(f'ORCL Volume', fontsize=16)
ax2.set_ylabel('Volume', fontsize=14)
ax2.set_xlabel('Date', fontsize=14)
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## Volume Spike Analysis

Let's analyze what happens after volume spikes.

In [ ]:
# Function to calculate forward returns
def calculate_forward_returns(df, days=[1, 3, 5, 10]):
    for day in days:
        df[f'return_{day}d'] = df['close'].pct_change(periods=day).shift(-day) * 100
    return df

# Calculate forward returns
df = calculate_forward_returns(df)

# Analyze returns after volume spikes
spike_returns = df[df['is_volume_spike']].copy()

# Display average returns after spikes
print("Average returns after volume spikes:")
for day in [1, 3, 5, 10]:
    avg_return = spike_returns[f'return_{day}d'].mean()
    print(f"{day}-day return: {avg_return:.2f}%")

# Display recent volume spikes
recent_spikes = spike_returns.tail(5)
recent_spikes[['date', 'close', 'volume', 'volume_z_score', 'return_1d', 'return_3d', 'return_5d']]

## Signal Generation

Based on volume analysis, let's generate trading signals.

In [ ]:
# Generate signals based on volume spikes
def generate_signals(df, direction):
    signals = []
    
    # Get recent volume spikes
    recent_spikes = df[df['is_volume_spike']].tail(3)
    
    for _, row in recent_spikes.iterrows():
        date = row['date']
        price_change = row['price_change']
        volume_z = row['volume_z_score']
        
        # For buy candidates
        if direction == 'buy':
            # Volume spike with positive price change
            if price_change > 0 and volume_z > 2.5:
                signals.append({
                    'date': date,
                    'signal': 'STRONG_BUY',
                    'confidence': min(volume_z / 5, 1.0),
                    'reason': f"Volume spike (Z={volume_z:.2f}) with positive price change ({price_change:.2f}%)"
                })
            # Volume spike with negative price change (potential reversal)
            elif price_change < 0 and volume_z > 3.0:
                signals.append({
                    'date': date,
                    'signal': 'POTENTIAL_REVERSAL',
                    'confidence': min(volume_z / 6, 0.8),
                    'reason': f"Volume spike (Z={volume_z:.2f}) with negative price change ({price_change:.2f}%)"
                })
        
        # For short candidates
        elif direction == 'short':
            # Volume spike with negative price change
            if price_change < 0 and volume_z > 2.5:
                signals.append({
                    'date': date,
                    'signal': 'STRONG_SHORT',
                    'confidence': min(volume_z / 5, 1.0),
                    'reason': f"Volume spike (Z={volume_z:.2f}) with negative price change ({price_change:.2f}%)"
                })
            # Volume spike with positive price change (potential reversal)
            elif price_change > 0 and volume_z > 3.0:
                signals.append({
                    'date': date,
                    'signal': 'POTENTIAL_REVERSAL',
                    'confidence': min(volume_z / 6, 0.8),
                    'reason': f"Volume spike (Z={volume_z:.2f}) with positive price change ({price_change:.2f}%)"
                })
    
    return signals

# Generate signals
signals = generate_signals(df, DIRECTION)

# Display signals
if signals:
    print(f"Generated {len(signals)} signals for ORCL:")
    for signal in signals:
        print(f"Date: {signal['date'].strftime('%Y-%m-%d')}")
        print(f"Signal: {signal['signal']}")
        print(f"Confidence: {signal['confidence']:.2f}")
        print(f"Reason: {signal['reason']}")
        print("---")
else:
    print(f"No signals generated for ORCL")

## Save Results to BigQuery

Let's save our analysis results to BigQuery for the master notebook to use.

In [ ]:
# Prepare analysis results for BigQuery
def prepare_analysis_results(df, signals, ticker, direction):
    # Get the latest data point
    latest = df.iloc[-1]
    
    # Get the latest signal if available
    latest_signal = None
    latest_confidence = 0.0
    notes = ""
    
    if signals:
        latest_signal = signals[0]['signal']
        latest_confidence = signals[0]['confidence']
        notes = signals[0]['reason']
    
    # Create results dataframe
    results = pd.DataFrame({
        'date': [latest['date']],
        'ticker': [ticker],
        'direction': [direction],
        'signal': [latest_signal if latest_signal else 'NEUTRAL'],
        'confidence': [latest_confidence],
        'volume_score': [latest['volume_z_score']],
        'is_spike': [latest['is_volume_spike']],
        'spike_strength': [latest['volume_ratio'] if latest['is_volume_spike'] else 0],
        'notes': [notes],
        'notebook_url': [f"https://colab.research.google.com/drive/your-notebook-id-for-{ticker}"],
        'timestamp': [datetime.now()]
    })
    
    return results

# Prepare analysis results
results = prepare_analysis_results(df, signals, TICKER, DIRECTION)

# Display results
results

In [ ]:
# Save analysis results to BigQuery
def save_to_bigquery(results, ticker):
    # Save to stock-specific analysis table
    analysis_table = f"{DATASET}.stock_{ticker}_analysis"
    results.to_gbq(analysis_table, PROJECT_ID, if_exists='append')
    
    # Save to master summary table
    summary_table = f"{DATASET}.master_summary"
    results.to_gbq(summary_table, PROJECT_ID, if_exists='append')
    
    print(f"Saved analysis results to BigQuery tables: {analysis_table} and {summary_table}")

# Uncomment to save results to BigQuery
# save_to_bigquery(results, TICKER)

## Summary

This notebook analyzed volume patterns and inefficiencies for ORCL.

### Key Findings:
- Number of volume spikes in the past year: 0
- Latest volume Z-score: 0.0
- Current signal: NEUTRAL

### Next Steps:
- Monitor for new volume spikes
- Check the master summary notebook for a comparison with other stocks